# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract text from a technical PDF
2. Prepare training data for causal language modelling (CLM)
3. Fine-tune a language (llama 3.2 3b) base model using QLoRA
4. Apply the QLoRA and the learned weights to the instruct model
5. Answer some test questions

In [1]:
# Clone the git repo to access the utilities
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

Cloning into 'mistralpeft'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 113 (delta 69), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 7.66 MiB | 23.15 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [2]:
# Make sure to be on the repo directory and pull
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git pull 

/kaggle/working/mistralpeft
Already up to date.


In [3]:
# Install the required packages from pypi
!pip install -r requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.6 MB/s eta 0:00:00


In [4]:
# Load packages
from transformers import Trainer, TrainingArguments, AutoTokenizer, pipeline
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftConfig, PeftModel
from datasets import Dataset

from mistralpeft.utils import TextExtractor, CLMPreprocessor

In [5]:
# Login to HuggingFace using Kaggle's secrets to be able to download models
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
login(secret_value_0) 

## 1. Extract Sentences from PDF
Several PDFs from my former research group at university (Thermo-Fluiddynamics Group, Prof. Polifke) are chosen to form the corpus

In [6]:
# TextExtractor is a simple ETL class to acquire a text corpus
pdf_files = [
    "Dissertation.pdf",
]
    
pdf_urls = [
    "https://mediatum.ub.tum.de/doc/1360567/1360567.pdf",
    "https://mediatum.ub.tum.de/doc/1601190/1601190.pdf",
    "https://mediatum.ub.tum.de/doc/1597610/1597610.pdf"
    "https://mediatum.ub.tum.de/doc/1584750/1584750.pdf",
    "https://mediatum.ub.tum.de/doc/1484812/1484812.pdf",
    "https://mediatum.ub.tum.de/doc/1335646/1335646.pdf",
    "https://mediatum.ub.tum.de/doc/1326486/1326486.pdf",
    "https://mediatum.ub.tum.de/doc/1306410/1306410.pdf",
    "https://mediatum.ub.tum.de/doc/1444929/1444929.pdf",
]
    
with TextExtractor("output/processed_documents.json") as extractor:
    # Process local files
    extractor.process_documents(pdf_files)
        
    # Process URLs
    extractor.process_documents(pdf_urls, url_list=True)

Processing documents: 100%|██████████| 8/8 [01:58<00:00, 14.84s/it]


## 2. Prepare MCLM Training Data

In [7]:
# Specify the model and load the tokenizer
# Llama 3.2 3B with approx. 3 billion parameters
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [8]:
# Preprocess the corpus for Causal Language Modeling (CLM)
json_file_paths = ["output/processed_documents.json"]
preprocessor = CLMPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

Token indices sequence length is longer than the specified maximum sequence length for this model (152682 > 131072). Running this sequence through the model will result in indexing errors


In [9]:
# Split into training and test set
train_test_set = dataset.train_test_split(test_size=0.1)
print(f"Created {len(train_test_set['train'])} training examples and {len(train_test_set['test'])} test examples")

# Preview a training example
example = train_test_set["train"][0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids'][:256]))

Created 232 training examples and 26 test examples

Example input:
 = 0. (2.012) Each factor represents an operator known from the convectio n equation with the con- vective speeds ±(c 0 ∓ u 0). In this form, it can be seen that the solution of the 1-D wave equation consists of two perturbations traveling upstream and downstream with the speed c 0 relative to the mean ﬂuid motion. These two characteristic w aves, also known as Rie- mann invariants, are deﬁned as f = 1 ( p′ ρ0 c 0 + u′ ) and g = 1 ( p′ ρ0 c 0 − u′ ). (2.013) Accordingly, the acoustic velocity and pressure is given in t erms of f and g as u′ = f − g and p′ ρ0 c 0 = f + g, (2.014) respectively. In linear acoustics, equations are often transformed into frequency domain by the Fourier transform F. Quantities in the frequency domain are indicated by ˆ· and the angular frequency is denoted as ω in the following. The pressure in frequency domain ˆ p is given by ˆp(ω) = F{p


## 3. Load and Prepare Model

In [10]:
# Load the base model
# Q4_K_M quantization of the base model is achieved through BitsAndBytes. It requires CUDA!
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors
    )

base_model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.float16, device_map="auto",
    quantization_config=quantization_config,
    )

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [11]:
# Configure the (Q)LoRA adaptor to use a rank of r=4
model = prepare_model_for_kbit_training(base_model)
config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

## 4. Train the Model

The LoRA adapter has about 6M parameters to train (compared to 3B parameters of the full LLM)

In [12]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=3, # Creates a virtual batch size of 3
    learning_rate=3e-4,
    fp16=True, # numerical precision of adapter is float16
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit", # Memory efficient optimizer
    log_level="info",
    report_to="none",
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training
trainer.train()

Using auto half precision backend
***** Running training *****
  Num examples = 232
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 3
  Gradient Accumulation steps = 3
  Total optimization steps = 231
  Number of trainable parameters = 6,078,464
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,3.140600,3.374806
2,9.963200,3.130455



***** Running Evaluation *****
  Num examples = 26
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-78
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rop

TrainOutput(global_step=231, training_loss=10.991053908418268, metrics={'train_runtime': 12827.5659, 'train_samples_per_second': 0.054, 'train_steps_per_second': 0.018, 'total_flos': 4.783239204333158e+16, 'train_loss': 10.991053908418268, 'epoch': 2.969827586206897})

In [13]:
# Save the LoRA adapter weights:
lora_save_path = "lora_weights"  # Directory to save the LoRA weights
model.save_pretrained(lora_save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.0",
  "u

## 5. Apply the learned weights to the instruct model

In [14]:
# Load the instruct model
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors
    )
model_name = "meta-llama/Llama-3.2-3B-Instruct"
instruct_tokenizer = AutoTokenizer.from_pretrained(model_name)
instruct_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config,
)

# Load the LoRA configuration and weights
lora_weights_path = "lora_weights"
peft_config = PeftConfig.from_pretrained(lora_weights_path)

# Apply LoRA adapter to the instruct model
lora_instruct_model = PeftModel.from_pretrained(instruct_model, lora_weights_path)  

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/tokenizer.json
loading file tokenizer.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/tokenizer_config.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Instantiating LlamaForCausalLM model under default dtype torch.float16.
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Llama-3.2-3B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}



## 6. Test the Model

In [15]:
def test_qanda(model, tokenizer):
    # Example queries about heat transfer and fluid dynamics
    queries = [
        "Explain the CFD/SI method.",
        "What is a Rijke tube?",
        "How is a finite impulse response computed?",
        "When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?",
        "For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude peaks in the frequency response at Strouhal numbers between 0-1, and how does this behavior change with Reynolds number?",
    ]

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    
    # Generate responses
    for query in queries:
        messages = [{"role": "user", "content": query}]
        prompt = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        outputs = pipe(prompt, max_new_tokens=120, do_sample=True)
        
        print(f"\nQuery: {query}")
        print(f"\nResponse: {outputs[0]['generated_text']}")
        print("-" * 80)

### 6.1 Answers to the test questions by the instruct model w/o LoRA

In [16]:
test_qanda(instruct_model, instruct_tokenizer)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.

Response: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Explain the CFD/SI method.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

CF/D ( Comible Flow/D Dynamics is a numerical technique used simulate fluid dynamics problems particularly in a steady steady flow. It based the Nav-Stu-L equations which describe conservation momentum mass and energy across a computational domain The is by discret the domain into small cells then discret the equations using finite differences finite difference. The method used solve for unknown at each, the result used compute velocity pressure and other properties the domain
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: What is a Rijke tube?

Response: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is a Rijke tube?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A Rijke tube, also known as aijke tube, is a device used in physics to demonstrate the principle of acoustic reson. It is a long, narrow tube with one end closed and other open. When air is blown through the tube, it creates aacical wave that resonates within the tube. The tube is typically made of glass or metal and has a diameter of about -  mm.
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: How is a finite impulse response computed?

Response: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

How is a finite impulse response computed?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A finite impulse is a signal that is equal to a Dirac delta function at instant t =0, and zero everywhere else. it is often used a reference signal for systems and is a useful tool signal processing. is computed by conv the input signal a the impulse.
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?

Response: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

 when analyzing heat transfer using system identification, key criteria used validate identified models include:

1. Root Mean Squ Error (MSE): measures difference between predicted actual output. smaller MSE generally better
2 Validation Data: data used validate model must representative the system being, not contaminated noise or artifacts
3 Cross Validation: validation 

### 6.2 Answers to the test questions by the adapted model

In [17]:
test_qanda(lora_instruct_model, instruct_tokenizer)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor


Query: Explain the CFD/SI method.

Response: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Explain the CFD/SI method.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

FDI, not C/S, is an method that is used to the the of incompress flow,. is a method used solve problems fluid, particularly the of flow around and around objects such as, and other geometries. method the the the of problems fluid, it used solve problems fluid, particularly the of flow around and around objects such as, and geomet.
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: What is a Rijke tube?

Response: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is a Rijke tube?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A Rijke tube is a device used in physics to produce a sound at a specific frequency. It is essentially a tube, typically made of metal, that is closed at one end and open at other, connected to a piston that oscillates back and forth within the tube. The tube is closed at one end by a orifice, which a piston pushes through to create a oscillating flow of air through the tube. The oscill flow creates a standing wave within the tube. When the piston pushes air through theifice, it creates a wave that travels the length of the tube, then reflected back.
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: How is a finite impulse response computed?

Response: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

How is a finite impulse response computed?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

a finite impulse response (IR) is a mathematical that describes how an system responds to an impulse, which is a sudden change in input. is computed by conv the impulse with system output. impulse is often represented the Kronecker delta,δt, which 1 everywhere t 0 and0 elsewhere.

IR can be computed several ways:

1 Conv with input and output: can conv the impulse with system outputx to getIRIRx. can also conv impulse inputx to getIRx.
2 Linear algebra: can compute IR using matrix. is matrix that has system and impulse as rows
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?

Response: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

 in system identification, the validation of identified models typically follows a set of criteria to ensure that the models accurately capture the underlying dynamics of system. key criteria for validation include:

1. data: validation requires a sufficient dataset that represents the system well, including exc and response. dataset should be long enough to capture releva